In [20]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

import transformers

from gcsl.algo.dt import DecisionTransformer, GPT2Model
from gcsl.tools import dotdict
import numpy as np

import einops

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
gpt_config = transformers.GPT2Config()
gpt_config.vocab_size=0 # doesn't matter -- we don't use the vocab
gpt_config.n_embd=8
gpt_config.n_layer=2
gpt_config.n_head=1
gpt_config.n_inner=4*gpt_config.n_embd
gpt_config.activation_function='relu'
gpt_config.n_positions=1024
gpt_config.resid_pdrop=0.1
gpt_config.attn_pdrop=0.1
gpt_config.n_ctx = gpt_config.n_positions # number of positions in attention mask
gpt = GPT2Model(gpt_config)

In [26]:
sum((p.numel() for p in gpt.parameters()))

1760

In [6]:
t = 10
X = torch.rand(32,10,64)

In [7]:
gpt.eval()
out = gpt(inputs_embeds=X)

In [8]:
X_s = X 
X_g = torch.rand(32,64)

In [9]:
X_g_r = einops.repeat(X_g, 'b h -> b repeat h', repeat=t)
X_g_r.shape

torch.Size([32, 10, 64])

In [19]:
X_sg, ps = einops.pack((X_s,X_g_r), 'b t *')
(X_sg[:,:, :64] == X_s).all(),(X_sg[:,:, 64:] == X_g_r).all()

(tensor(True), tensor(True))